In [27]:
import os
os.chdir(r"\Users\Akhil\OneDrive\Desktop\animal-disease-classification-with-llm")

In [28]:
import tensorflow as tf
import json
model = tf.keras.models.load_model("artifacts/training/trained_model.h5")

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [30]:
from classifier.constants import *
from classifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([Path(self.config.artifacts_root)])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/trained_model.h5",
            training_data="artifacts/data_ingestion/unzip/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [32]:
from urllib.parse import urlparse

In [ ]:
from classifier.utils.common import save_json
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path_to_json=Path("scores.json"), data=scores)

In [38]:
try:
    config = ConfigurationManager()
    eval_config = config.get_validation_config()

    evaluation = Evaluation(config=eval_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

created directory at: artifacts


Found 116 images belonging to 2 classes.
8/8 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9397 - loss: 0.7452


TypeError: save_json() missing 1 required positional argument: 'self'